# Robust Portfolio

This tutorial shows how to construct and solve the Robust Portfolio optimization problem. The problem can be formulated as follows:
$$
\begin{align}
\max_{\mathbf{x}} &\left( \mathbf{p}+\mathbf{\Delta}\mathbf{z} \right)^T \mathbf{x} \nonumber \\

\text{s.t. } &\sum_{i=1}^n x_i = 1 \nonumber \\

&\mathbf{x}\geq \mathbf{0} \nonumber \\

&\mathbf{z}\in \left\{\mathbf{z}:  \left \lVert \mathbf{z} \right \rVert_{\infty} \leq 1, \left \lVert \mathbf{z} \right \rVert_{1} \leq \Gamma \right\} \nonumber \\
\end{align}
$$

where each stock has a value in an interval $\left[ p_i - \delta_i, p_i + \delta_i \right]$, $\Delta$ is a diagonal matrix with $\left\{ \delta_i \right\}_i$ on its diagonal, and $\Gamma$ is the budget of the uncertainty parameter.

In [1]:
import cvxpy as cp
import numpy as np

from lropt import Budget
from lropt.robust_problem import RobustProblem
from lropt.uncertain import UncertainParameter

np.random.seed(seed=1234)

We start by defining the relevant constants as shown below:

In [2]:
n = 150                                             # Number of stocks
i = np.arange(1, n+1)                               # Indices vector
p = 1.15 + i*0.05/150                               # Mean of the returns
delta = np.array(0.05/450 * (2*i*n*(n+1))**0.5)     # Deviations of the returns
Gamma = 5                                           # Uncertainty budget

The problem is formulated and solved in the block below, using the Budget uncertainty set which is defined as follows:

Budget uncertainty: $\{u \mid \| A_1u + b_1 \|_\infty \leq \rho_1, \| A_2u + b_2 \|_1 \leq \rho_2\}$ 

- $\rho_1$ : float, optional  
  * Box scaling. Default 1.0. 
- $\rho_2$ : float, optional
  * 1-norm scaling. Default 1.0.
- $A_1$, $A_2$ : np.array, optional
  * Scaling matrix for u. Default identity matrix.
- $b_1$, $b_2$ : np.array, optional
  * Relocation vector for u. Default None.

In [3]:
x = cp.Variable(n)                                                  # Optimization variables, investment fractions
uncertainty_set = Budget(rho1=1, rho2=Gamma)                        # Budget uncertainty set
z = UncertainParameter(n, uncertainty_set=uncertainty_set)          # Uncertain parameters, control the random return
objective = cp.Maximize((p + cp.multiply(delta,z)) @ x)             # Problem objective
constraints = [
                cp.sum(x)==1,
                x>=0,
              ]                                                     # Problem constraints

prob = RobustProblem(objective=objective, constraints=constraints)  # Define the problem using LROPT
prob.solve(solver="CLARABEL")                                       # Solve the problem using LROPT

1.1708896495314707